## From TensorFlow to ONNX

In [ ]:
import tensorflow as tf
import onnx
import tf2onnx

# Convert the model to ONNX format
loaded_model = tf.keras.models.load_model('../devil-in-gan/data/art-dgm-ipynb-data/benign-dcgan-mnist')
loaded_model.summary()

onnx_model, _ = tf2onnx.convert.from_keras(loaded_model)

model_path = './benign-dcgan-mnist.onnx'
onnx.save(onnx_model, model_path)

2024-11-06 00:25:11.634235: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 00:25:11.833969: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 00:25:11.834010: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 00:25:11.835176: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 00:25:11.943172: I tensorflow/core/platform/cpu_feature_g

2024-11-06 00:25:14.017291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 00:25:14.087661: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3136)              313600    
                                                                 
 batch_normalization (Batch  (None, 3136)              12544     
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3136)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 7, 7, 32)          51200     
 anspose)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 7, 7, 32)         

2024-11-06 00:25:14.403967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 00:25:14.404011: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-11-06 00:25:14.404229: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-06 00:25:14.404581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 00:25:14.404595: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

## From ONNX to pytorch

In [24]:
import torch
import onnx
import onnxruntime as ort
from onnx2pytorch import ConvertModel


# Cargar el modelo ONNX
model_path = './benign-dcgan-mnist.onnx'
onnx_model = onnx.load(model_path)

# Convert ONNX model to PyTorch
pytorch_model = ConvertModel(onnx_model)

torch.save(pytorch_model.state_dict(), "benign-dcgan-mnist.pth")

## ConvertModel from tensorflow -> onnx -> pytorch

In [ ]:
import torch
import onnx
import numpy as np
from onnx2pytorch import ConvertModel
from torch.autograd import Variable

is_cuda_available = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if is_cuda_available else torch.FloatTensor

# Cargar el modelo ONNX
onnx_model = onnx.load("benign-dcgan-mnist.onnx")

# Convertir el modelo ONNX a PyTorch
pytorch_model = ConvertModel(onnx_model)

# model = TheModelClass(*args, **kwargs)
pytorch_model.load_state_dict(torch.load("benign-dcgan-mnist.pth", weights_only=True))
pytorch_model.eval()

print(pytorch_model)


# z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))
# pytorch_model(z)

ConvertModel(
  (MatMul_sequential_2/dense/MatMul:0): Linear(in_features=100, out_features=3136, bias=False)
  (Mul_sequential_2/batch_normalization/batchnorm/mul_1:0): tanh()
  (Add_sequential_2/batch_normalization/batchnorm/add_1:0): Add()
  (LeakyRelu_sequential_2/leaky_re_lu/LeakyRelu:0): LeakyReLU(negative_slope=0.30000001192092896, inplace=True)
  (Reshape_sequential_2/reshape/Reshape:0): Reshape(shape=[-1  7  7 64])
  (Transpose_sequential_2/conv2d_transpose/conv2d_transpose__37:0): Transpose()
  (ConvTranspose_sequential_2/conv2d_transpose/conv2d_transpose:0): ConvTranspose2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (BatchNormalization_sequential_2/batch_normalization_1/FusedBatchNormV3:0): BatchNormWrapper(
    (bnu): BatchNormUnsafe(32, eps=0.0010000000474974513, momentum=0.1, affine=True, track_running_stats=True)
  )
  (LeakyRelu_sequential_2/leaky_re_lu_1/LeakyRelu:0): LeakyReLU(negative_slope=0.30000001192092896, inplace=True)
  (ConvTransp

NotImplementedError: Input with larger batch size than 1 not supported yet.